# Fast Semantic Chunking Demo

**Approach**: Agno's native semantic chunking
- PDFReader for text extraction
- SemanticChunking (natural boundaries)
- PgVector hybrid search (vector + FTS)
- Gemini embeddings (text-embedding-004)

**Best for**: Fast prototyping

## Setup

In [1]:
import subprocess
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
pdf_dir = project_root / "data" / "pdfs"

print(f"✅ Project root: {project_root}")

✅ Project root: /home/vitor/contextual-rag-agno-supabase


## 1. Download Sample PDFs

In [2]:
# if pdf_dir.exists() and list(pdf_dir.glob("*.pdf")):
#     print(f"✅ PDFs already downloaded: {len(list(pdf_dir.glob('*.pdf')))} files")
# else:
#     print("📥 Downloading sample PDFs...")
#     result = subprocess.run([sys.executable, str(project_root / "scripts" / "download_pdfs.py")], 
#                           capture_output=True, text=True, cwd=str(project_root))
#     print(result.stdout)

## 2. Initialize Knowledge Base

In [3]:
from src.storage.agno_knowledge import AgnoKnowledge

kb = AgnoKnowledge(table_name="economics_docs_gemini")
print("✅ Knowledge base initialized")
print("   - Semantic chunking: ON")
print("   - Hybrid search: ON")
print(f"   - Table: {kb.knowledge.vector_db.table_name}")

✅ Knowledge base initialized
   - Semantic chunking: ON
   - Hybrid search: ON
   - Table: economics_docs_gemini


## 3. Ingest PDFs

In [4]:
kb.ingest_directory(str(pdf_dir))
print("✅ All PDFs ingested")

INFO Adding content from path, 6cdcf639-e8d3-54a0-9c63-ece7291a494f, None,                                         
     /home/vitor/contextual-rag-agno-supabase/data/pdfs, None

INFO Adding content from path, 944a1797-17db-5bbc-846b-481e3d7185ac, None,                                         
     /home/vitor/contextual-rag-agno-supabase/data/pdfs/O-Homem-Mais-Rico-da-Babilonia-George-S.-Clason.pdf, None

INFO Processing batch starting at index 0, size: 100

INFO Upserted batch of 100 documents.

INFO Processing batch starting at index 100, size: 20

INFO Upserted batch of 20 documents.

✅ All PDFs ingested


## 4. Search with Hybrid Search

In [6]:
results = kb.search("O que é a quinta lei de outro?", limit=3)

print("Search Results:\n" + "="*80)
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result.content[:300]}...")
    print("-"*80)

INFO Found 3 documents

Search Results:

1. AS CINCO LEIS DE OURO I. O ouro vem de bom grado e numa quantidade crescente para todo homem que separa não menos de um décimo de seus ganhos, a fim de criar um fundo para o seu futuro e o de sua própria família. II.0 ouro trabalha diligente e satisfatoriamente para o homem prudente que, possuindo-o...
--------------------------------------------------------------------------------

2. ganhan do. Esse é o resultado da primeira lei." A Segunda Lei de Ouro O ouro trabalha diligente e satisfatoriamente para o homem pru dente que, possuindo-o, encontra para ele um emprego lucrativo, multiplicando-o como os flocos de algodão no campo. "O ouro realmente é um trabalhador bem-disposto. Es...
--------------------------------------------------------------------------------

3. que nove.' Debatam o assunto entre vocês. Se alguém puder provar que isso não é verdade, conversaremos a respeito amanhã quando estivermos juntos de novo."...
-------------------------------------------

## 5. Query with LLM Agent

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini

agent = Agent(
    model=Gemini(id="gemini-2.5-flash"),
    knowledge=kb.knowledge,
    search_knowledge=True,
    markdown=True,  # Opcional: formata a resposta em markdown
    debug_mode=True,  # Opcional: mostra detalhes de execução
)

agent.print_response(
    "Explique o que é a quinta lei de outro",
    stream=True
)

DEBUG ****** Agent ID: d473b6c7-bff3-4752-8c1c-9ae463e21bd8 ******

DEBUG Creating new AgentSession: 793db008-b99d-49ea-93b3-d9b1bd2a8aeb

DEBUG Processing tools for model

DEBUG Added tool search_knowledge_base

DEBUG ** Agent Run Start: 8c838078-7480-44c2-892b-659eeed2270e ***

...

DEBUG * Tokens:                      input=18512, output=746, total=19258

DEBUG ************************  METRICS  *************************

DEBUG ---------------- Google Response Stream End ----------------

DEBUG Added RunOutput to Agent Session

DEBUG *** Agent Run End: 8c838078-7480-44c2-892b-659eeed2270e ****

## 6. Try Different Queries

In [ ]:
# queries = [
#     "What are the principles of wealth building?",
#     "How does money and credit work in the economy?",
#     "What is the role of government in economics?"
# ]

# for query in queries:
#     print(f"\n{'='*80}")
#     print(f"Q: {query}")
#     print('='*80)
#     agent.print_response(query, stream=True)
#     print("\n")